# 4.5 Выделение контуров на изображении
### Оператор Шарра


In [1]:
import os
import numpy as np
from scipy.signal import convolve2d
from PIL import Image as pim

opp_Gx = np.array([[3, 0, -3],
                [10, 0, -10],
                [3, 0, -3]])

opp_Gy = np.array([[3, 10, 3],
                [0, 0, 0],
                [-3, -10, -3]])

BLACK = 0
WHITE = 255

In [9]:
def sharr_function(input_path, output_path):

    #1
    input_image = pim.open(input_path).convert('RGB')
    input_array = np.array(input_image)

    H, W = input_array.shape[:2]
    
    #2
    semitone_array = np.zeros((H, W), dtype=np.uint8)

    for y in range(H):
        for x in range(W):
            semitone_array[y, x] = np.mean(input_array[y, x])

    #3: AFTER SEMITONE OF INPUT IMAGE:
    Gx = convolve2d(semitone_array, opp_Gx, mode='same')
    Gy = convolve2d(semitone_array, opp_Gy, mode='same')
    G = np.sqrt(Gx**2 + Gy**2)                             # Gradient Formula

    G_max = np.max((np.max(Gx), np.max(Gy), np.max(G)))
    G = G * WHITE / G_max
    Gx = Gx * WHITE / G_max
    Gy = Gy * WHITE / G_max

    G_path = output_path + '/' + 'G_' + os.path.basename(input_path)
    pim.fromarray(Gx.astype(np.uint8), mode='L').save(output_path + '/' + 'Gx_' + os.path.basename(input_path))
    pim.fromarray(Gy.astype(np.uint8), mode='L').save(output_path + '/' + 'Gy_' + os.path.basename(input_path))
    pim.fromarray(G.astype(np.uint8), mode='L').save(G_path)
    return G_path


In [1]:
#4
def binarize(input_path, output_path, threshold):
    input_image = pim.open(input_path)    
    input_array = np.array(input_image)
    pim.fromarray(np.where(input_array < threshold, BLACK, WHITE).astype(np.uint8), mode='L').save(output_path)

In [12]:
def main():
    images = ["im1.png", "im2.png", "im3.png"]

    for image in images:
        output_path = "output/" + image
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        G_path = sharr_function('input/' + image, output_path)
        binarize(G_path, output_path + '/' + "bin_" + os.path.basename(G_path), 27)

if __name__ == "__main__":
    main()